In [1]:
import torch
from torch import nn 
import matplotlib.pyplot as plt


In [6]:
def showplt(x, y,title, x_name, y_name):
    # Plot the signal
	plt.figure(figsize=(10, 5))
	plt.plot(x.numpy(), y.numpy())
	plt.title(title)
	plt.xlabel(x_name)
	plt.ylabel(y_name)
	plt.grid(True)
	plt.show()

In [5]:
T = 1000
time = torch.arange(1, T + 1, dtype=torch.float32)
x = torch.sin(0.01 * time) + torch.normal(0, 0.2, (T,))
# sin(0.01x) + 噪声 
showplt(time, x, 'x vs. time', 'time', 'x')

torch.Size([1000])